In [0]:
file_path = "/Volumes/sample_catalog/sample_schema/sample_volume/Data Engineering and ML Workshop (Responses) - Form Responses 1.csv"

print(file_path)

In [0]:


df = spark.read.csv(file_path, header=True, inferSchema=True, multiLine=True)
df.display()


In [0]:
df.printSchema()


In [0]:
short_col_names = {
    "What is your full name?": "Name",
    "What is your current education level?": "Degree",
    "What is your age?": "Age",
    "What is your gender?": "Gender",
    "Which programming languages do you know?": "Langs",
    "How familiar are you with Python?": "PyLvl",
    "\n\nHow would you rate your data engineering knowledge?": "DE_Rate",
    "\n\nWhat do you want to learn from this workshop?": "Expect",
    "What is your zip code?": "Zip"
}

df_short = df
for old, new in short_col_names.items():
    df_short = df_short.withColumnRenamed(old, new)

display(df_short)

In [0]:
from pyspark.sql.functions import regexp_replace, col

df_clean = df_short.withColumn("Age", regexp_replace(col("Age"), "[^0-9]", ""))
display(df_clean)

In [0]:
from pyspark.sql.functions import trim, col

df_clean = df_clean.select([trim(col(c)).alias(c) for c in df_clean.columns])


In [0]:
df_clean.groupBy("Degree").count().display()


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
df_clean.withColumn("DE_Rate", col("DE_Rate").cast("float")).groupBy("Degree").avg("DE_Rate").display()


In [0]:
df_clean.filter(col("Age").isNotNull()).select(col("Age").cast("int").alias("Age")).display()

Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import split, explode

df_langs = df_clean.withColumn("Language", explode(split(col("Langs"), ",\\s*")))
df_langs.groupBy("Langs").count().orderBy("count", ascending=False).display()


Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import lower, split, explode

df_words = (
    df_clean
    .select(explode(split(lower(col("Expect")), "\\s+")).alias("word"))
)

df_words.groupBy("word").count().orderBy("count", ascending=False).display()


Databricks visualization. Run in Databricks to view.

In [0]:
display(df_clean)

In [0]:
df_pincode = spark.read.format("csv").option("header", "true").load("/Volumes/sample_catalog/sample_schema/sample_volume/pincode_with_lat-long.csv")
display(df_pincode)

Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import regexp_replace, length

df_clean2 = df_clean.withColumnRenamed("What is your pin code?\r", "pincode")
df_clean2 = df_clean2.withColumn("pincode", regexp_replace("pincode", r"^\s+|\s+$", ""))
df_clean2 = df_clean2.withColumn("pincode_length", length("pincode"))
display(df_clean2)

In [0]:
from pyspark.sql.functions import col

df_joined = df_clean2.join(df_pincode, df_clean2.pincode == df_pincode.pincode, "left") \
    .withColumn("latitude", col("latitude").cast("float")) \
    .withColumn("longitude", col("longitude").cast("float"))
display(df_joined)

Databricks visualization. Run in Databricks to view.